In [2]:
import numpy 
import pandas
import nltk
import csv
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

##### Los topicos se encuentran juntos en el dataframe, esta función los separa de acuerdo al tema

In [29]:
def separaTopico(topico, df): #Separa todos los encabezados de acuerdo a su tópico
    listTopic = []
    dataArr = numpy.asarray(df)
    for i in range(len(dataArr)):
        if(dataArr[i][1] == topico):
            listTopic.append(dataArr[i][0])
    return listTopic

##### Cuando se tienen las palabras unicas de cada tópico, se eliminan caracteres especiales.

In [79]:
def limpiaTexto(text):
    "List all the word tokens in a text."
    return re.findall('[a-zA-Z0-9\-]{4,254}', text.lower())


##### Cada encabezado corresponde a un documento, esta función separa cada palabra para agregarlas a un solo array

In [19]:
def PalabrasTopico(listTopico): #Devuelve todas las palabras que aparecen en los titulares de las noticias 
    listHeadLines = []
    for i in range(len(listTopico)):
        #aux = nltk.word_tokenize(arrayTopico[i])
        aux = limpiaTexto(listTopico[i])
        for j in range(len(aux)):
            aux2 = aux[j]
            listHeadLines.append(aux2)
            
    return numpy.asarray(listHeadLines)

In [170]:
def Probabilidad(palabrasTopico, palabrasUnicasTopico): #Devuelve un array con la ocurrencias de cada única del tópico
    Prob = numpy.zeros(len(palabrasUnicasTopico))
    for i in range(len(palabrasUnicasTopico)):
        Prob[i] = numpy.sum(palabrasTopico == palabrasUnicasTopico[i])
        Prob[i] = Prob[i] /len(palabrasTopico)
        
    return Prob

##### Recibe el vector de probabilidades y elige la máxima para finalmente devolver el tópico al que pertenece

In [306]:
def Clasificador(arrayProbabilidades):
    indice = numpy.where(arrayProbabilidades == max(arrayProbabilidades))[0][0]
    print("Indice = ", type(indice))
    print("Max = ", max(arrayProbabilidades))
    if (indice == 0):
        print("Covid")
        return topico
    if (indice == 1):
        print("Tecnología")
        return topico
    if (indice == 2):
        print("Deportes")

In [299]:
def MAP(tweet, palabrasCovid, AlfabetoCovid, palabrasTecn, AlfabetoTecn, palabrasDep, AlfabetoDep):
    T = nltk.word_tokenize(tweet)
    Proba = numpy.ones(3)

    #Covid
    for j in range(len(T)):
        aux = numpy.sum(palabrasCovid == T[j]) + 1
        Proba[0] *= (aux + 1) / (len(AlfabetoCovid) + aux)
    
    #Tecnologia
    for j in range(len(T)):
        aux = numpy.sum(palabrasTecn == T[j]) 
        Proba[1] *= (aux + 1) / (len(AlfabetoTecn) + aux)

    #Deportes
    for j in range(len(T)):
        aux = numpy.sum(palabrasDep == T[j]) + 1
        Proba[2] *= (aux + 1) / (len(AlfabetoDep) + aux)

    for p in range(len(Proba)):
        Proba[p] = Proba[p] * 1/3
        
    return Clasificador(Proba)
        

##### Abrimos el dataset con pandas

In [286]:
nombres = ['Titular', 'Tópico']
dataTrain = pandas.read_csv('titulares.csv', names=nombres)
dataTrain

,Titular,Tópico
0,Que política o economía conduzcan respuesta al...,covid
1,Oaxaca habilita red de hospitales para covid-19,covid
2,Surgen más brotes de covid-19 en asilo y empre...,covid
3,Suman cuatro mil 477 muertos y 42 mil 595 cont...,covid
4,Revelan que más de mil 400 mujeres han solicit...,covid
...,...,...
1195,"En los Parapanamericanos, México suma ya 42 or...",deportes
1196,El futbolista Jonathan Fabbro es condenado a 1...,deportes
1197,México cosecha 50 medallas en el paraatletismo...,deportes
1198,Debut y gol de Edson Álvarez que da pase al Aj...,deportes


##### Separamos del dataset por tópicos y a cada encabezado lo separamos por palabra de los documentos eliminando caracteres especiales

In [287]:
palabrasCovid = PalabrasTopico(separaTopico('covid', dataTrain))
AlfabetoCovid = numpy.unique(palabrasCovid)
#print(len(palabrasCovid))
#print(len(AlfabetoCovid))

palabrasTecn = PalabrasTopico(separaTopico('tecnologia', dataTrain))
AlfabetoTecn = numpy.unique(palabrasTecn)
print(len(palabrasTecn))
print(AlfabetoTecn)


palabrasDep = PalabrasTopico(separaTopico('deportes', dataTrain))
AlfabetoDep = numpy.unique(palabrasDep)

Alfabeto = numpy.concatenate((AlfabetoCovid, AlfabetoTecn, AlfabetoDep))
#Alfabeto.shape

1399
['2018' '2019' '2021' '30-50' 'abacus' 'abierto' 'abre' 'abrir'
 'aburridos' 'abusivas' 'acad' 'acciones' 'acepta' 'acoso' 'acotar'
 'activista' 'acusa' 'acusaciones' 'acusan' 'afecta' 'afectados' 'agota'
 'agradable' 'ahora' 'airpods' 'alan' 'alexa' 'algo' 'algoritmo'
 'algoritmos' 'algunos' 'alista' 'alphabet' 'alternativa' 'alternativas'
 'amazon' 'amenaza' 'amigo' 'amigos' 'amlo' 'analytica' 'ancestro'
 'android' 'androide' 'anel' 'anonymous' 'ante' 'antig' 'antiguas'
 'antiguos' 'antitabaco' 'anuncia' 'anuncian' 'anunciantes' 'aparecer'
 'aplicaci' 'aplicaciones' 'aplicar' 'apocalipsis' 'apoyar' 'apple' 'apps'
 'aprendi' 'apuesta' 'apunta' 'archivos' 'aretes' 'artificial'
 'artificiales' 'arya' 'asesinato' 'asesino' 'asilo' 'asistente'
 'assassin' 'ataca' 'audios' 'aumenta' 'aunque' 'autorretratos' 'auxilio'
 'avatar' 'avecina' 'ayuda' 'baja' 'bajo' 'banner' 'barcelona' 'batalla'
 'batallas' 'battle' 'bernie' 'bicicleta' 'bien' 'bill' 'billete'
 'bitcoin' 'bitos' 'blackberry'

##### Obtenemos las probabilidades a priori, haciendo cada una de estas equiprobabloe $ P(C) $

In [288]:
PCovid = 1/3
PTecn = 1/3
PDep = 1/3
print("PCovid = ", PCovid, "PTecn = ", PTecn, "PDep = ", PDep)

PCovid =  0.3333333333333333 PTecn =  0.3333333333333333 PDep =  0.3333333333333333


##### Obtenemos la frecuencia de palabras de cada atributo (palabra) $ P(A_{i} | C_{j}) $

In [289]:
ProbaCov = Probabilidad(palabrasCovid, AlfabetoCovid)
print(ProbaCov)
ProbaTecn = Probabilidad(palabrasTecn, AlfabetoTecn)
ProbaDep = Probabilidad(palabrasDep, AlfabetoDep)

[0.00060976 0.00030488 0.00030488 ... 0.00182927 0.00030488 0.00030488]


##### Pruebas

In [307]:
tweet = "fútbol medalla jugador"

In [308]:
MAP(tweet, palabrasCovid, AlfabetoCovid, palabrasTecn, AlfabetoTecn, palabrasDep, AlfabetoDep)

Indice =  <class 'numpy.int64'>
Max =  5.3324306594712984e-09
Deportes
